In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end

#####################################

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:17<00:00, 37.11it/s] 


{0: [(0.07823637778985218+0j) [Z6 Z7]],
 1: [(-4.087119674570706+0j) []],
 2: [(0.07823637778985248+0j) [Z8 Z9]],
 3: [(-0.22781796302889024+0j) [Z9],
  (-0.004217284878422751+0j) [Y6 Y7 X8 X9],
  (-0.0019065797888500567+0j) [Y0 Z1 Z2 X3 X8 Y9],
  (0.006069919256437085+0j) [Y2 X3 X8 Y9],
  (-0.0048588716929696615+0j) [Y3 Z4 Z5 Z6 Z7 Y8 X9 X10],
  (-0.004797924342406355+0j) [Y3 X4 X8 Y9]],
 4: [(-0.22781796302888996+0j) [Z6],
  (-0.004217284878422751+0j) [X6 X7 Y8 Y9],
  (0.004821390001231589+0j) [Y6 X7 X10 Y11],
  (0.004858871692969653+0j) [X2 Z3 Z4 Z5 X6 Y7 Z8 Z9 Z10 Y11],
  (-0.0014751106104476744+0j) [X0 Z1 Z2 Z3 Z4 Z5 X6 Y7 Z8 Z9 Z10 Y11],
  (0.0034750158150631504+0j) [X4 Z5 X6 Y7 Z8 Z9 Z10 Y11]],
 5: [(-0.22781796302889018+0j) [Z8],
  (0.004217284878422751+0j) [X6 Y7 Y8 X9],
  (-0.010337447912440532+0j) [X4 X5 Y8 Y9],
  (0.002559511780295127+0j) [Y1 Z2 Z3 Y4 Y8 Y9],
  (-0.004797924342406355+0j) [Y3 Y4 Y8 Y9],
  (0.003475015815063157+0j) [X4 Z5 Z6 Z7 X8 X9 Z10 X11]],
 6: [(-0.22781

In [6]:
NOON_spins_combined

[1.9809621905738606,
 1.9809621905738604,
 0.034585473061157654,
 4.917485196617946e-05,
 0.0017204854695842902,
 0.001720485469584279]

In [2]:
from quchem.Ansatz_Generator_Functions import *

##
NMO_basis, NOON_spins_combined = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NMO_basis=NMO_basis,
    occ_threshold= 0.999,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [1^ 3^ 5 9] +
 1.0 [9^ 5^ 3 1],
 -1.0 [1^ 3^ 5 11] +
 1.0 [11^ 5^ 3 1],
 -1.0 [1^ 3^ 9 11] +
 1.0 [11^ 9^ 3 1]]

In [3]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()

print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 3


In [7]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

full_ansatz_Q_Circ = Ansatz_Circuit(Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab,
             Hamilt.molecule.n_qubits, Hamilt.molecule.n_electrons)

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, reduced_theta_parameters_ijab)

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

0 1         2 3         4 5           6 7 8 9         10 11
│ │         │ │         │ │           │ │ │ │         │  │
X X         X X         │ H           │ │ │ H         │  H
│ │         │ │         │ │           │ │ │ │         │  │
│ Rx(0.5π)  │ │         │ │           │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ @─────────X │         │ │           │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ │         @─X         │ │           │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ │         │ @─────────X │           │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ │         │ │         @─X           │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ │         │ │         │ Rz(0.0π)    │ │ │ │         │  │
│ │         │ │         │ │           │ │ │ │         │  │
│ │         │ │         @─X           │ │ │ │         │

In [8]:

simulator = cirq.Simulator()
result = simulator.compute_amplitudes(ansatz_cirq_circuit, bitstrings=[i for i in range(2 ** len(ansatz_cirq_circuit.all_qubits()))])
result=np.around(result, 5)
print(result.reshape([(2 ** len(ansatz_cirq_circuit.all_qubits())), 1]))

[[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 ...
 [-0.+0.j]
 [-0.+0.j]
 [-0.+0.j]]


In [9]:
from quchem.LCU_method import *

In [10]:
from quchem.Simulating_Quantum_Circuit import *
R_uncorrected, Pn, gamma_l = Get_R_linear_combination(anti_commuting_sets[10], 1)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)

full_circ = Full_Q_Circuit(Pn, R_corrected_Op_list, R_corr_list, ancilla_amplitudes, Hamilt.molecule.n_qubits, ansatz_cirq_circuit)
output_bin_dict = Get_binary_dict_project(full_circ, Pn, 100000, Hamilt.molecule.n_qubits, ancilla_amplitudes, l1)
expectation_value_by_parity(output_bin_dict) * gamma_l 

(0.06557255450176665+0j)

In [11]:
n_shots=10


testing =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 n_shots,
                 Hamilt.molecule.n_qubits,
                 N_indices_dict=None)
testing.Calc_Energy()

(-7.749033259592099+0j)

In [13]:
Hamilt.molecule.fci_energy

array(-7.88098231)

In [14]:
n_shots=10

def GIVE_ENERGY(theta_ia_theta_jab_list):
    theta_ia = theta_ia_theta_jab_list[:len(reduced_theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(reduced_theta_parameters_ia):]
   
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_ia, theta_ijab)

    VQE_exp_LCU =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                     ansatz_cirq_circuit,
                     n_shots,
                     Hamilt.molecule.n_qubits,
                     N_indices_dict=None)#{7:0, 8:1, 9:0, 10:1})
    return VQE_exp_LCU.Calc_Energy().real


In [15]:
### optimizer
from quchem.Scipy_Optimizer import *
THETA_params=[0 for _ in range(len(reduced_theta_parameters_ia)+len(reduced_theta_parameters_ijab))]

GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

0: Input_to_Funct: [0.      0.00025 0.      0.      0.      0.      0.      0.      0.     ]: Output: -7.880548956069913
1: Input_to_Funct: [0.      0.00025 0.      0.      0.      0.      0.      0.      0.     ]: Output: -7.900081482127246
2: Input_to_Funct: [0.      0.00025 0.      0.      0.      0.      0.      0.      0.     ]: Output: -7.900225854923956


KeyboardInterrupt: 

### Tensorflow

In [16]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [17]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY(new_theta_list)
        
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [18]:
import random

X0 = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))]

GG = Tensor_Flow_Optimizer(GIVE_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
GG.optimize(50)
GG.plot_convergence()


starting input: [1.6927027419518608, 1.535147308687081, 5.021782430638702, 2.9079068232022296, 1.547600555021634, 4.530582338372917, 3.304528731559831, 5.691022731808912, 3.527365684355265] obj funct out: -5.683321763754747
step 0 INPUT: [1.7927027, 1.4351474, 4.9217825, 2.807907, 1.6476004, 4.630582, 3.4045286, 5.7910223, 3.4273658] OUTPUT: -5.694728424854979
step 1 INPUT: [1.8927469, 1.3419687, 4.822021, 2.716511, 1.7473774, 4.573001, 3.49867, 5.8725057, 3.3322575] OUTPUT: -6.020107864576309
step 2 INPUT: [1.9833915, 1.2669199, 4.7935805, 2.6580653, 1.8473676, 4.57627, 3.5912654, 5.887003, 3.234934] OUTPUT: -5.994694620937441
step 3 INPUT: [2.0434463, 1.253892, 4.7397647, 2.5968876, 1.9225421, 4.6164937, 3.6867983, 5.840432, 3.2412379] OUTPUT: -6.453726822356894
step 4 INPUT: [2.0324836, 1.2350212, 4.6714487, 2.5760465, 1.9559247, 4.5867257, 3.7832522, 5.7833138, 3.28551] OUTPUT: -6.443249366240502
step 5 INPUT: [2.0012846, 1.2123758, 4.606797, 2.5233274, 2.0050664, 4.554651, 3.8811

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/lex/anaconda3/envs/QuantumUpdated/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-f6d3d918ba1d>", line 7, in <module>
    GG.optimize(50)
  File "/home/lex/Documents/PhD/VQE-code/quchem/TensorFlow_Opt.py", line 152, in optimize
    grads_and_vars = self.grad_funct(input_vect)
  File "<ipython-input-17-88f32740fa07>", line 10, in calc_gradient
    Obs_PLUS = GIVE_ENERGY(new_theta_list)
  File "<ipython-input-14-d98c6cd4ec8b>", line 14, in GIVE_ENERGY
    return VQE_exp_LCU.Calc_Energy().real
  File "/home/lex/Documents/PhD/VQE-code/quchem/LCU_method.py", line 690, in Calc_Energy
    binary_state_counter = self.Get_binary_dict_project(Q_circuit, Pn, ancilla_amplitudes, l1_norm)
  File "/home/lex/Documents/PhD/VQE-code/quchem/LCU_method.py", line 653, in Get_binary_dict_project
    raw_result = simulate_probabilistic_Q_circu

KeyboardInterrupt: 